In [1]:
import os
import pyaudio
import wave
from tensorflow import keras
import  numpy as np
import time

In [2]:
p = pyaudio.PyAudio()
for i in range(p.get_device_count()):
    print(p.get_device_info_by_index(i))


{'index': 0, 'structVersion': 2, 'name': 'HDA NVidia: HDMI 0 (hw:0,3)', 'hostApi': 0, 'maxInputChannels': 0, 'maxOutputChannels': 8, 'defaultLowInputLatency': -1.0, 'defaultLowOutputLatency': 0.005804988662131519, 'defaultHighInputLatency': -1.0, 'defaultHighOutputLatency': 0.034829931972789115, 'defaultSampleRate': 44100.0}
{'index': 1, 'structVersion': 2, 'name': 'HDA NVidia: HDMI 1 (hw:0,7)', 'hostApi': 0, 'maxInputChannels': 0, 'maxOutputChannels': 8, 'defaultLowInputLatency': -1.0, 'defaultLowOutputLatency': 0.005804988662131519, 'defaultHighInputLatency': -1.0, 'defaultHighOutputLatency': 0.034829931972789115, 'defaultSampleRate': 44100.0}
{'index': 2, 'structVersion': 2, 'name': 'HDA NVidia: HDMI 2 (hw:0,8)', 'hostApi': 0, 'maxInputChannels': 0, 'maxOutputChannels': 2, 'defaultLowInputLatency': -1.0, 'defaultLowOutputLatency': 0.005804988662131519, 'defaultHighInputLatency': -1.0, 'defaultHighOutputLatency': 0.034829931972789115, 'defaultSampleRate': 44100.0}
{'index': 3, 'struc

In [4]:
SPEAKERS = p.get_default_output_device_info()["hostApi"] #The modified part

In [ ]:
p = pyaudio.PyAudio()

info = p.get_host_api_info_by_index(0)
numdevices = info.get('deviceCount')
for i in range (0,numdevices):
        if p.get_device_info_by_host_api_device_index(0,i).get('maxInputChannels')>0:
                print("Input Device id ", i, " - ", p.get_device_info_by_host_api_device_index(0,i).get('name'))

        if p.get_device_info_by_host_api_device_index(0,i).get('maxOutputChannels')>0:
                print("Output Device id ", i, " - ", p.get_device_info_by_host_api_device_index(0,i).get('name'))


In [ ]:
files = os.listdir("/home/andrea/Scrivania/MIVIA_ROAD_DB1/noised_dataset/")
for i,f in enumerate(files):
    files[i] = "/home/andrea/Scrivania/MIVIA_ROAD_DB1/noised_dataset/"+f

In [21]:
from pydub import AudioSegment

audio = AudioSegment.from_file("/home/andrea/Scrivania/audio_only.wav").get_array_of_samples()


MODEL TEST

In [5]:

model = keras.models.load_model("audio_model_bidirectional_LSTM_overlap.h5")

dataset = np.load("dataset_overlap.npy")
std_dev = np.std(dataset)
mean = np.mean(dataset)
maximum = np.max(dataset)
min = np.min(dataset)
scale = max(maximum, abs(min))

def normalize(array, std_dev, mean, scale):
    array = array/scale
    return array-mean/std_dev

In [6]:
CHUNK = 1024
FORMAT = pyaudio.paInt16
CHANNELS = 1
RATE = 44100

In [7]:
SPEAKERS = p.get_default_output_device_info()["hostApi"] #The modified part
frames = []
do_prediction = False
window = np.zeros((1,32,16516))
filled = 0
p = pyaudio.PyAudio()
stream = p.open(format=FORMAT,
                channels=CHANNELS,
                rate=RATE,
                input=True,
                input_host_api_specific_stream_info=SPEAKERS,
                frames_per_buffer=CHUNK)
add = False
start = time.time()

while True:
    st1 = time.time()
    data = stream.read(CHUNK,exception_on_overflow = False)
    frames.append(data)
    if CHUNK*len(frames) == 16384:

        s_arr = np.zeros((1,1,16516), dtype=np.int16)
        s_arr[0,0,:16384] = np.frombuffer(b''.join(frames), dtype=np.int16)
        del frames[:int(CHUNK/2*len(frames))]
        add = True
    if filled <= 31 and add:
        window[0,filled,:] = normalize(s_arr[0,0,:], std_dev, mean, scale)
        filled += 1

    elif filled > 31 and add:
        window = np.hstack((np.delete(window, 0, 1), normalize(s_arr, std_dev, mean, scale)))
        print("Full Window")
        do_prediction = True
    add = False
    if do_prediction:
        c = np.argmax(model.predict(window), axis=-1)
        #do_prediction = False
        print(c)

Full Window
Instructions for updating:
Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
[[0]]
[[0]]
[[0]]
[[0]]
[[0]]
[[0]]
[[0]]
[[0]]
[[0]]
[[0]]
[[0]]
[[0]]
[[0]]
[[0]]
[[0]]
[[0]]
Full Window
[[0]]
[[0]]
[[0]]
[[0]]
[[0]]
[[0]]
[[0]]
[[0]]
[[0]]
[[0]]
[[0]]
[[0]]
[[0]]
[[0]]
[[0]]
[[0]]
Full Window
[[0]]
[[0]]
[[0]]
[[0]]
[[0]]
[[0]]
[[0]]
[[0]]
[[0]]
[[0]]
[[0]]
[[0]]
[[0]]
[[0]]
[[0]]
[[0]]
Full Window
[[0]]
[[0]]
[[0]]
[[0]]
[[0]]
[[0]]
[[0]]
[[0]]
[[0]]
[[0]]
[[0]]
[[0]]
[[0]]
[[0]]
[[0]]
[[0]]
Full Window
[[0]]
[[0]]
[[0]]
[[0]]
[[0]]
[[0]]
[[0]]
[[0]]
[[0]]
[[0]]
[[0]]
[[0]]
[[0]]
[[0]]
[[0]]
[[0]]
Full Window
[[0]]
[[0]]
[[0]]
[[0]]
[[0]]
[[0]]
[[0]]
[[0]]
[[0]]
[[0]]
[[0]]
[[0]]
[[0]]
[[0]]
[[0]]
[[0]]
Full Window
[[0]]


KeyboardInterrupt: 

In [31]:
stream.stop_stream()
stream.close()
p.terminate()

MATRIX TEST

In [7]:
array = np.random.randint(100, size=(1,16,16516))

In [8]:
array_post = (np.delete(array, 0, 1))

In [8]:
start = time.time()
model.predict_classes(window)
print("Inference Time: " +str(time.time()-start))

Inference Time: 0.08124518394470215


In [ ]:
model.evaluate()

FAKE CHECKER

In [32]:
RECORD_SECONDS = 25

SPEAKERS = p.get_default_output_device_info()["hostApi"] #The modified part
frames = []
windows = []
filled = 0
p = pyaudio.PyAudio()
stream = p.open(format=FORMAT,
                channels=CHANNELS,
                rate=RATE,
                input=True,
                input_host_api_specific_stream_info=SPEAKERS,
                frames_per_buffer=CHUNK)

for i in range(0, int(RATE / CHUNK * RECORD_SECONDS) + 1):
    data = stream.read(CHUNK)
    frames.append(data)
    '''
    if CHUNK*len(frames) == 16384:

        s_arr = np.zeros((1,1,16516))
        s_arr[0,0,:16384] = np.frombuffer(b''.join(frames), dtype=np.int16)
        frames = []
        windows.append(s_arr)
    '''

In [33]:
data = np.frombuffer(b''.join(frames), dtype=np.int16)

In [35]:
limit = int(len(data)/8258)*8258
test_data = data[:limit]

In [36]:
sliding_windows = []
step = 16516
half_step = int(step/2)
for j in range(int(len(test_data)/half_step)-1):
    sliding_windows.append(test_data[j*half_step:j*half_step+step])

In [37]:
fake_ds = []

for j in range(len(sliding_windows)-32):
    fake_ds.append(sliding_windows[j:j+32])

In [38]:
np_ds = np.asarray(fake_ds)

In [39]:
model.predict_classes(normalize(np_ds, std_dev, mean, scale))

array([[0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [1],
       [0],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
    

In [28]:
m1 = np.max(np.asarray(audio))
m2 = np.max(np_ds)